In [1]:
!pip install torch torchaudio librosa soundfile scikit-learn tqdm
!pip install panns-inference

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 73.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
!wget -O Cnn14_mAP=0.431.pth "https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth?download=1"

--2025-08-06 06:52:59--  https://zenodo.org/record/3987831/files/Cnn14_mAP%3D0.431.pth?download=1
Resolving zenodo.org (zenodo.org)... 188.185.48.194, 188.185.43.25, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.48.194|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/3987831/files/Cnn14_mAP=0.431.pth [following]
--2025-08-06 06:52:59--  https://zenodo.org/records/3987831/files/Cnn14_mAP=0.431.pth
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 327428481 (312M) [application/octet-stream]
Saving to: ‘Cnn14_mAP=0.431.pth’

Cnn14_mAP=0.431.pth 100%[===================>] 312.26M  27.4MB/s    in 2m 23s  

2025-08-06 06:55:23 (2.18 MB/s) - ‘Cnn14_mAP=0.431.pth’ saved [327428481/327428481]



In [3]:
# === Dependencies ===
import os
import random
import numpy as np
import librosa
import xml.etree.ElementTree as ET
from glob import glob
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, classification_report

# === Config / hyperparams ===
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SR = 32000
WINDOW_SECONDS = 1.0
WINDOW_SAMPLES = int(SR * WINDOW_SECONDS)
N_MELS = 64
FFT_SIZE = 1024
HOP_LENGTH = 512
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
NUM_EPOCHS = 30
SEED = 42
FOLDS = ["A", "B", "C", "D"]
LABEL_MAP = {2: 0, 3: 1}  # 0=skid,1=crash, background=2

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

# === Utility functions ===
def load_audio(path, sr=SR):
    y, _ = librosa.load(path, sr=sr)
    return y

def pad_or_trim(y, target_len=WINDOW_SAMPLES):
    if len(y) < target_len:
        pad_len = target_len - len(y)
        y = np.pad(y, (0, pad_len))
    elif len(y) > target_len:
        y = y[:target_len]
    return y

def extract_log_mel(y, sr=SR, n_mels=N_MELS, n_fft=FFT_SIZE, hop_length=HOP_LENGTH):
    S = librosa.feature.melspectrogram(
        y=y, sr=sr, n_fft=n_fft, hop_length=hop_length,
        n_mels=n_mels, fmin=50, fmax=14000
    )
    S_db = librosa.power_to_db(S, ref=np.max)
    return S_db


# === Dataset class ===
class MIVIARoadAudioDataset(Dataset):
    def __init__(self, root_dir, folds, window_sec=WINDOW_SECONDS, sr=SR, augment=False, neg_per_pos=1):
        self.sr = sr
        self.window_samples = int(window_sec * sr)
        self.augment = augment
        self.examples = []  # (audio_path, center_sample, label)
        self._prepare(root_dir, folds, neg_per_pos)

    def _parse_xml(self, xml_path):
        try:
            tree = ET.parse(xml_path)
            root = tree.getroot()
            events = []
            ev_root = root.find("events")
            if ev_root is None:
                return events
            for item in ev_root.findall("item"):
                try:
                    start = float(item.find("STARTSECOND").text)
                    end = float(item.find("ENDSECOND").text)
                    cid = int(float(item.find("CLASS_ID").text))
                    if cid not in LABEL_MAP:
                        continue
                    events.append({"start": start, "end": end, "class_id": cid})
                except:
                    continue
            return events
        except Exception as e:
            print(f"[XML parse error] {xml_path}: {e}")
            return []

    def _compute_non_event_intervals(self, events, total_dur):
        sorted_ev = sorted(events, key=lambda x: x["start"])
        gaps = []
        curr = 0.0
        for ev in sorted_ev:
            if ev["start"] > curr:
                gaps.append((curr, ev["start"]))
            curr = max(curr, ev["end"])
        if curr < total_dur:
            gaps.append((curr, total_dur))
        return gaps

    def _sample_negative_center(self, gaps):
        lengths = [e - s for s, e in gaps]
        if not lengths:
            return 0.0
        probs = [l / sum(lengths) for l in lengths]
        idx = np.random.choice(len(gaps), p=probs)
        start, end = gaps[idx]
        min_c = start + WINDOW_SECONDS / 2.0
        max_c = end - WINDOW_SECONDS / 2.0
        if min_c >= max_c:
            return (start + end) / 2.0
        return random.uniform(min_c, max_c)

    def _prepare(self, root_dir, folds, neg_per_pos):
        for fold in folds:
            fold_dir = os.path.join(root_dir, fold)
            v2_dir = os.path.join(fold_dir, "v2")
            if not os.path.isdir(fold_dir) or not os.path.isdir(v2_dir):
                print(f"[Warning] Missing fold or v2: {fold_dir} / {v2_dir}")
                continue
            xml_files = glob(os.path.join(fold_dir, "*.xml"))
            for xml_path in xml_files:
                base = os.path.splitext(os.path.basename(xml_path))[0]
                wav_candidates = glob(os.path.join(v2_dir, f"{base}*.wav"))
                if not wav_candidates:
                    print(f"[Warning] No WAV for XML {base} in {v2_dir}")
                    continue
                audio_path = wav_candidates[0]
                y = load_audio(audio_path, sr=self.sr)
                audio_dur = len(y) / self.sr
                events = self._parse_xml(xml_path)
                positive_centers = []
                for ev in events:
                    center = (ev["start"] + ev["end"]) / 2.0
                    center_sample = int(center * self.sr)
                    label = LABEL_MAP[ev["class_id"]]
                    self.examples.append((audio_path, center_sample, label))
                    positive_centers.append((max(0, ev["start"]), min(audio_dur, ev["end"])))
                gaps = self._compute_non_event_intervals(events, audio_dur)
                for _ in range(len(positive_centers) * neg_per_pos):
                    center_time = self._sample_negative_center(gaps)
                    center_sample = int(center_time * self.sr)
                    self.examples.append((audio_path, center_sample, 2))  # background label

        if len(self.examples) == 0:
            print("[Error] No examples loaded. Check dataset path/structure.")
        else:
            print(f"[Info] Loaded {len(self.examples)} examples from folds {folds}.")

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        audio_path, center_sample, label = self.examples[idx]
        y = load_audio(audio_path, sr=self.sr)
        start = max(0, center_sample - self.window_samples // 2)
        end = start + self.window_samples
        if end > len(y):
            end = len(y)
            start = end - self.window_samples
            if start < 0:
                start = 0
        segment = y[start:end]
        segment = pad_or_trim(segment, self.window_samples)
        feat = extract_log_mel(segment, sr=self.sr)  # (n_mels, time)
        if self.augment and random.random() < 0.3:
            noise = np.random.randn(*segment.shape) * 0.005
            segment = segment + noise
            feat = extract_log_mel(segment, sr=self.sr)
        feat_tensor = torch.tensor(feat, dtype=torch.float32).unsqueeze(0)  # (1, n_mels, time)
        target = torch.tensor(label, dtype=torch.long)
        return feat_tensor, target

# === RCNN Model ===
class RCNNClassifier(nn.Module):
    def __init__(self, n_mels=64, n_classes=3, hidden_size=128):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Conv2d(16, 32, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d((2,2)),
        )
        self.lstm = nn.LSTM(input_size=32 * (n_mels // 4), hidden_size=hidden_size, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 64), nn.ReLU(),
            nn.Linear(64, n_classes)
        )

    def forward(self, x):  # x: (B,1,n_mels,time)
        x = self.cnn(x)  # (B, C, M', T')
        B, C, M, T = x.size()
        x = x.permute(0, 3, 1, 2).contiguous()  # (B, T, C, M)
        x = x.view(B, T, C * M)  # (B, T, feat)
        x, _ = self.lstm(x)  # (B, T, hidden)
        x = x[:, -1, :]  # last timestep
        out = self.classifier(x)  # (B, n_classes)
        return out

# === Training / evaluation helpers ===
def train_one_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0.0
    all_preds, all_targets = [], []
    for X, y in tqdm(loader, desc="Training", leave=False):
        X = X.to(DEVICE)
        y = y.to(DEVICE)
        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X.size(0)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_targets.extend(y.cpu().numpy())
    avg_loss = total_loss / len(loader.dataset)
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average="weighted", zero_division=0)
    return avg_loss, precision, recall, f1

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0.0
    all_preds, all_targets = [], []
    with torch.no_grad():
        for X, y in tqdm(loader, desc="Evaluating", leave=False):
            X = X.to(DEVICE)
            y = y.to(DEVICE)
            logits = model(X)
            loss = criterion(logits, y)
            total_loss += loss.item() * X.size(0)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_targets.extend(y.cpu().numpy())
    avg_loss = total_loss / len(loader.dataset)
    precision, recall, f1, _ = precision_recall_fscore_support(all_targets, all_preds, average="weighted", zero_division=0)
    cm = confusion_matrix(all_targets, all_preds)
    return avg_loss, precision, recall, f1, cm, all_targets, all_preds

# === Cross-validation runner ===
def run_cross_validation(root_dir):
    fold_scores = []
    for i in range(len(FOLDS)):
        val_fold = FOLDS[i]
        train_folds = [f for f in FOLDS if f != val_fold]
        print(f"\n=== CV Fold: train on {train_folds}, validate on {val_fold} ===")
        train_ds = MIVIARoadAudioDataset(root_dir, train_folds, augment=True)
        val_ds = MIVIARoadAudioDataset(root_dir, [val_fold], augment=False)
        if len(train_ds) == 0 or len(val_ds) == 0:
            print("[Warning] empty split, skipping fold.")
            continue
        train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
        val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

        model = RCNNClassifier(n_mels=N_MELS, n_classes=3).to(DEVICE)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

        best_f1 = 0.0
        for epoch in range(NUM_EPOCHS):
            tr_loss, tr_p, tr_r, tr_f1 = train_one_epoch(model, train_loader, criterion, optimizer)
            val_loss, val_p, val_r, val_f1, cm, tgts, preds = evaluate(model, val_loader, criterion)
            print(f"[Fold {val_fold}] Epoch {epoch+1}/{NUM_EPOCHS} train_f1={tr_f1:.4f} val_f1={val_f1:.4f}")
            if val_f1 > best_f1:
                best_f1 = val_f1
                torch.save(model.state_dict(), f"best_rcnn_fold_{val_fold}.pt")
        fold_scores.append(best_f1)
        # report per-fold confusion matrix summary
        print(f"Confusion matrix for fold {val_fold}:\n{cm}")
        print("Classification report:\n", classification_report(tgts, preds, zero_division=0))
    if fold_scores:
        print(f"\nCross-validation F1 scores: {fold_scores}")
        print(f"Mean CV F1: {np.mean(fold_scores):.4f}")

# === Entry point for Kaggle ===
# Adjust this path if your dataset root differs
dataset_root = "/kaggle/input/mivia-dataset/MIVIA_ROAD_DB1/audio"  # or "/kaggle/input/mivia-dataset" if A/B/C/D are directly there
run_cross_validation(dataset_root)


=== CV Fold: train on ['B', 'C', 'D'], validate on A ===
[Info] Loaded 600 examples from folds ['B', 'C', 'D'].
[Info] Loaded 200 examples from folds ['A'].


[Fold A] Epoch 1/30 train_f1=0.3624 val_f1=0.3333


[Fold A] Epoch 2/30 train_f1=0.3333 val_f1=0.3333


[Fold A] Epoch 3/30 train_f1=0.3656 val_f1=0.6590


[Fold A] Epoch 4/30 train_f1=0.7034 val_f1=0.7458


...

[Fold D] Epoch 26/30 train_f1=1.0000 val_f1=0.9062


[Fold D] Epoch 27/30 train_f1=1.0000 val_f1=0.9103


[Fold D] Epoch 28/30 train_f1=1.0000 val_f1=0.9103


[Fold D] Epoch 29/30 train_f1=1.0000 val_f1=0.9103


[Fold D] Epoch 30/30 train_f1=1.0000 val_f1=0.9202
Confusion matrix for fold D:
[[47  3  0]
 [ 0 42  8]
 [ 0  5 95]]
Classification report:
               precision    recall  f1-score   support

           0       1.00      0.94      0.97        50
           1       0.84      0.84      0.84        50
           2       0.92      0.95      0.94       100

    accuracy                           0.92       200
   macro avg       0.92      0.91      0.92       200
weighted avg       0.92      0.92      0.92       200


Cross-validation F1 scores: [0.9899474641341685, 0.9851042334039721, 0.9601262979478054, 0.9461159012919651]
Mean CV F1: 0.9703
